This aim of this notebook is to figure out how to get data from spotify. 

Resources : https://medium.com/@rafaelnduarte/how-to-retrieve-data-from-spotify-110c859ab304


## récupération des crédits 

Pour pouvoir utiliser l'api, il faut d'abord se créer un compte https://developer.spotify.com/dashboard/ puis créer une application. Une fois créée, il faut aller dans '''edit settings''' et rajouter les url 'http://localhost/callback/' et 'http://localhost/callback' dans les Redirected uri. 

## utilisation

In [1]:
# ! pip install spotipy

In [2]:
import spotipy 
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
import spotipy.util as util

In [3]:
# setting up authorization
cid ="your_cliend" 
secret = "your_secret"
# saving the info you're going to need
username = 'your_user'
scope = 'user-library-read' #check the documentation
authorization_url = 'https://accounts.spotify.com/authorize'
token_url = 'https://accounts.spotify.com/api/token'
redirect_uri ='http://localhost/callback/'

token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri=redirect_uri)
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# retrieving you access token
auth = SpotifyClientCredentials(
client_id=cid,
client_secret=secret)
# save your token
token = auth.get_access_token()
spotify = spotipy.Spotify(auth=token)
# check if everything is in order
print(token)
print(spotify)

{'access_token': 'BQBHzVBvWB0zVgipOSb5oYm_anBeaj5ETIoC8qxryR3M06QyUAxS7l32nXOHu10LDpz2OyqblVpMYE1_9Mw', 'token_type': 'Bearer', 'expires_in': 3600, 'expires_at': 1645369142}


C:\Users\Elise\AppData\Local\Temp/ipykernel_11376/2615581724.py:6: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = auth.get_access_token()


In [6]:
# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,2000,50):
    print(i)
    track_results = sp.search(q='year:2022', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'year:2022', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': None} returned 404 due to Not found.


1000


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/search?q=year%3A2022&limit=50&offset=1000&type=track:
 Not found., reason: None

In [10]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
# check if everything is ok
df_tracks.head()

(1000, 4)


,artist_name,track_name,track_id,popularity
0,Gunna,pushin P (feat. Young Thug),3XOalgusokruzA5ZBA2Qcb,92
1,Fabio Bevere,YEAR 2022,7niIWQiDqvFvyNtRJBgjIg,0
2,DJ Snake,U Are My High,4db4crB87tyZ9ExtEoxgVc,7
3,Years & Years,King,3ydd63Uoimtpjg1hgNSFTg,3
4,The Weeknd,Sacrifice,1nH2PkJL1XoUq8oE6tBZoU,93


In [12]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

100

In [13]:
# dropping duplicates

df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [14]:
# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    print(i)
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

0
100
200
300
400
500
600
700
800
Number of tracks where no audio features were available: 2


In [17]:
# saving the features in a data frame
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns') 
print("Shape of the dataset:", df_audio_features.shape) 
df_audio_features.head()

Shape of the dataset: (803, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.773,0.422,1,-4.572,0,0.1870,0.00783,0.006930,0.1290,0.488,77.502,audio_features,3XOalgusokruzA5ZBA2Qcb,spotify:track:3XOalgusokruzA5ZBA2Qcb,https://api.spotify.com/v1/tracks/3XOalgusokru...,https://api.spotify.com/v1/audio-analysis/3XOa...,136267,1
1,0.809,0.548,7,-10.460,1,0.1130,0.56400,0.844000,0.0964,0.104,120.003,audio_features,7niIWQiDqvFvyNtRJBgjIg,spotify:track:7niIWQiDqvFvyNtRJBgjIg,https://api.spotify.com/v1/tracks/7niIWQiDqvFv...,https://api.spotify.com/v1/audio-analysis/7niI...,308000,4
2,0.503,0.783,0,-4.761,0,0.0385,0.02210,0.002230,0.1400,0.213,125.122,audio_features,4db4crB87tyZ9ExtEoxgVc,spotify:track:4db4crB87tyZ9ExtEoxgVc,https://api.spotify.com/v1/tracks/4db4crB87tyZ...,https://api.spotify.com/v1/audio-analysis/4db4...,180480,4
3,0.559,0.837,4,-4.062,0,0.0391,0.06170,0.000000,0.3930,0.446,119.958,audio_features,3ydd63Uoimtpjg1hgNSFTg,spotify:track:3ydd63Uoimtpjg1hgNSFTg,https://api.spotify.com/v1/tracks/3ydd63Uoimtp...,https://api.spotify.com/v1/audio-analysis/3ydd...,214835,4
4,0.735,0.795,11,-6.523,0,0.1130,0.02960,0.000032,0.0678,0.905,122.000,audio_features,1nH2PkJL1XoUq8oE6tBZoU,spotify:track:1nH2PkJL1XoUq8oE6tBZoU,https://api.spotify.com/v1/tracks/1nH2PkJL1XoU...,https://api.spotify.com/v1/audio-analysis/1nH2...,188918,4


In [ ]:
df_audio_features.to_csv('audio_features_one.csv')

In [19]:

df_tracks.describe()

,popularity
count,805.000000
mean,16.609938
std,27.601047
min,0.000000
25%,0.000000
50%,4.000000
75%,14.000000
max,93.000000
